In [11]:
import torch
import torch.nn as nn

In [3]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        out = self.fc(hidden[-1])
        out = self.sigmoid(out)
        return out

In [5]:
input_size = 26
output_size = 4
hidden_size = 128

In [6]:
model = LSTM(input_size, hidden_size, output_size)

In [9]:
model_weights = torch.load('../models/100_epochs.pth', map_location=torch.device('cpu'))
model.load_state_dict(model_weights)
model.eval()

LSTM(
  (lstm): LSTM(26, 128, batch_first=True)
  (fc): Linear(in_features=128, out_features=4, bias=True)
  (sigmoid): Sigmoid()
)

### input
[samples][timesteps][13 x 2]  
e.g. [1000][9][26]  
### output
[samples][labels]  
e.g. [1000][4]  